In [23]:
from datetime import datetime
import pandas as pd
import numpy as np
import scipy
import einops
from thefuzz import process
from tqdm import tqdm
import requests
import json
import pickle

We first download the "Ausgrid average electricity consumption by LGA 2021 excel" file from https://www.ausgrid.com.au/Industry/Our-Research/Data-to-share/Average-electricity-use and save to the data folder

In [24]:
elec_con_data = pd.read_excel(
    "data/Ausgrid average electricity consumption by LGA 2021.xlsx",
    usecols="B:Q",
    skiprows=29,
    nrows=9,
    names=[
        "LGA", "Residential Daily average (kWh)", "Residential General Supply (MWh)", "Residential Off Peak Hot Water (MWh)", "Residential Total (MWh)", "Off Peak Residential Customer number", "Total Residential Customer number",
        "Number of Residential Solar Customers", "Number of Non-Residential Solar Customers", "Residential Generation Capacity (kWp)", "Non-Residential Generation Capacity (kWp)", "Energy exported to the grid (MWh)",
        "Non-Residential Small-Medium Sites MWh", "Number of Non-Residential Small-Medium Sites",
        "Non-Residential Large Sites MWh", "Number of Non-Residential Large Sites",
    ]
)
elec_con_data

,LGA,Residential Daily average (kWh),Residential General Supply (MWh),Residential Off Peak Hot Water (MWh),Residential Total (MWh),Off Peak Residential Customer number,Total Residential Customer number,Number of Residential Solar Customers,Number of Non-Residential Solar Customers,Residential Generation Capacity (kWp),Non-Residential Generation Capacity (kWp),Energy exported to the grid (MWh),Non-Residential Small-Medium Sites MWh,Number of Non-Residential Small-Medium Sites,Non-Residential Large Sites MWh,Number of Non-Residential Large Sites
0,CENTRAL COAST*,16.508265,749993.75163,175439.24757,925432.99920,80849.438356,153585.665753,27926,866,129951.235,25729.240,94069.939235,208282.67026,12157.939726,507257.00212,709.723288
1,CESSNOCK,18.009241,145062.19126,22340.42324,167402.61450,10649.386301,25466.775342,6119,269,34436.875,11315.320,32556.795154,38708.85680,1843.106849,73069.18906,135.123288
2,LAKE MACQUARIE,16.913623,433830.86633,106992.88358,540823.74991,53160.884932,87604.465753,19815,539,96355.520,12551.950,71404.042118,118244.94451,6795.572603,189172.40852,391.958904
3,MAITLAND,16.812069,186156.22587,27389.42481,213545.65068,13031.569863,34799.797260,8541,268,46886.140,8038.030,36534.991861,59161.52808,2959.734247,91106.93447,170.602740
4,MUSWELLBROOK,20.195877,43567.83267,10159.80349,53727.63616,4443.449315,7288.567123,1306,94,7703.790,1825.385,6401.202701,28306.77256,1347.082192,19973.42688,52.008219
5,NEWCASTLE,13.670967,311806.04897,53494.28002,365300.32899,29693.865753,73207.901370,11139,561,49787.295,20393.255,46486.583101,156675.75499,8035.923288,263849.58008,515.241096
6,PORT STEPHENS,16.915889,169997.82293,46769.07573,216766.89866,23513.298630,35107.934247,8151,245,40727.000,5935.905,31131.420972,53077.87271,2985.665753,89789.91025,194.471233
7,SINGLETON,21.464937,64190.69011,13978.25635,78168.94646,6087.583562,9977.271233,2447,144,15021.225,5561.590,12941.132405,27270.28062,1676.934247,32676.20314,92.002740
8,UPPER HUNTER,20.126945,34744.97519,7854.87092,42599.84611,3524.008219,5798.789041,1143,109,6751.025,1868.590,5986.931273,20682.52559,1251.331507,8937.49130,26.515068


Allocating residential usages according to five classes

In [25]:
def alloc_mean(x, allocation):
    return (1 / len(allocation)) * sum([x * a for a in allocation])

def f(x, allocation, mu):
    return sum(abs(alloc_mean(x, allocation) - mu))

N = 5
rng = np.random.default_rng(50)
allocation = rng.dirichlet([1.5 for i in range(N)])
mu0 = scipy.optimize.minimize(f, np.repeat(0, len(elec_con_data)), (allocation, elec_con_data.loc[:, "Residential Daily average (kWh)"]))['x']

print(f"{allocation=}, {mu0=}: {alloc_mean(mu0, allocation)=}")
print(f"{np.array([mu0 * a for a in allocation])=}")

allocation=array([0.15889974, 0.18347015, 0.36256272, 0.1651147 , 0.12995269]), mu0=array([ 82.54132513,  90.04620605,  84.56811482,  84.06034257,
       100.97938367,  68.35483605,  84.57944433, 107.32468435,
       100.63472418]): alloc_mean(mu0, allocation)=array([16.50826503, 18.00924121, 16.91362296, 16.81206851, 20.19587673,
       13.67096721, 16.91588887, 21.46493687, 20.12694484])
np.array([mu0 * a for a in allocation])=array([[13.11579506, 14.30831869, 13.43785142, 13.35716654, 16.04559776,
        10.86156564, 13.43965167, 17.05386439, 15.99083146],
       [15.14386933, 16.52079096, 15.51572474, 15.42256369, 18.52670271,
        12.54107205, 15.51780337, 19.69087597, 18.46346798],
       [29.92640729, 32.64739733, 30.66124567, 30.47714639, 36.61135994,
        24.78291523, 30.66535333, 38.9119294 , 36.48639925],
       [13.62878632, 14.8679525 , 13.9634391 , 13.87959843, 16.67318087,
        11.2863884 , 13.96530977, 17.7208833 , 16.61627252],
       [10.72646712, 11.7017465

Now we generate synthetic data based on these statistics

In [26]:
res_X = {}
for (i, elec_con_row), m0, in zip(elec_con_data.iterrows(), mu0):
    res_X[i] = rng.normal(
        loc=np.array([m0 * a for a in allocation]),
        scale=5,
        size=(365, (round(elec_con_row["Total Residential Customer number"]) - round(elec_con_row["Number of Residential Solar Customers"])) // N, N)
    ).T

Doing the same with small-medium non-residential but this time with only 2 classes (small and medium)

In [27]:
N = 2
allocation = rng.dirichlet([1.5 for i in range(N)])
non_res_small_med_daily_avg = elec_con_data.loc[:, "Non-Residential Small-Medium Sites MWh"] / 365 * 1000 / elec_con_data.loc[:, "Number of Non-Residential Small-Medium Sites"]
mu0 = scipy.optimize.minimize(f, np.repeat(0, len(elec_con_data)), (allocation, non_res_small_med_daily_avg))['x']

print(f"{allocation=}, {mu0=}: {alloc_mean(mu0, allocation)=}")
print(f"{np.array([mu0 * a for a in allocation])=}")

allocation=array([0.28585821, 0.71414179]), mu0=array([ 93.87074877, 115.07923421,  95.34406324, 109.52765683,
       115.14190001, 106.83243937,  97.41132562,  89.10677057,
        90.56665378]): alloc_mean(mu0, allocation)=array([46.93537438, 57.53961711, 47.67203162, 54.76382842, 57.57095001,
       53.41621969, 48.70566281, 44.55338528, 45.28332689])
np.array([mu0 * a for a in allocation])=array([[26.83372437, 32.89634409, 27.25488341, 31.30938011, 32.91425762,
        30.53893006, 27.84582733, 25.47190208, 25.88922168],
       [67.0370244 , 82.18289012, 68.08917983, 78.21827673, 82.22764239,
        76.29350931, 69.56549829, 63.63486849, 64.67743209]])


And the synthetic data for the small-medium sites

In [28]:
small_med_non_res_X = {}
for (i, elec_con_row), m0, in zip(elec_con_data.iterrows(), mu0):
    small_med_non_res_X[i] = rng.normal(
        loc=np.array([m0 * a for a in allocation]),
        scale=5,
        size=(365, (round(elec_con_row["Number of Non-Residential Small-Medium Sites"]) - round(5 * elec_con_row["Number of Non-Residential Solar Customers"] / 6)) // N, N)
    ).T
small_med_non_res_X

{0: array([[[41.24479284, 31.22395289, 21.20451162, ..., 20.61517204,
          28.1371749 , 31.92745303],
         [13.23286408, 22.23134103, 26.53537478, ..., 28.04931701,
          26.78131479, 19.60611236],
         [26.75050761, 21.08091539, 24.87864835, ..., 24.29742935,
          29.73932768, 29.76952906],
         ...,
         [25.20923738, 37.8312225 , 26.69571979, ..., 24.20486391,
          30.75450895, 24.45354621],
         [26.59252008, 22.81041156, 28.74165095, ..., 21.57218431,
          27.79640656, 27.9472027 ],
         [27.72050049, 27.38905267, 30.62166461, ..., 23.40248784,
          30.62604707, 22.47262834]],
 
        [[69.81623205, 68.68967738, 66.22087408, ..., 61.53019415,
          62.91581251, 72.67412508],
         [64.54939634, 62.89620952, 62.37189054, ..., 73.72171537,
          66.42211385, 66.38096043],
         [75.02911252, 74.83511539, 59.1273476 , ..., 70.36618341,
          73.60745647, 63.25544423],
         ...,
         [60.36235195, 71.5647

We will assign a single mean for large non-residential sites

In [29]:
non_res_large_daily_avg = elec_con_data.loc[:, "Non-Residential Large Sites MWh"] / 365 * 1000 / elec_con_data.loc[:, "Number of Non-Residential Large Sites"]
print(f"{non_res_large_daily_avg.tolist()=}")

non_res_large_daily_avg.tolist()=[1958.1507827476653, 1481.5326248986207, 1322.2829379652603, 1463.09514164124, 1052.174412895749, 1402.985063941339, 1264.9673191795107, 973.0562859950568, 923.4853585451541]


And finally, we generate the synthetic data

In [30]:
large_non_res_X = {}
for (i, elec_con_row), avg in zip(elec_con_data.iterrows(), non_res_large_daily_avg):
    large_non_res_X[i] = rng.normal(
        loc=avg,
        scale=5,
        size=(365, round(elec_con_row["Number of Non-Residential Large Sites"]) - round(elec_con_row["Number of Non-Residential Solar Customers"] / 6), 1)
    ).T
large_non_res_X

{0: array([[[1958.06378146, 1954.03774891, 1962.82816691, ...,
          1957.99788785, 1955.89187929, 1958.14769605],
         [1965.84920174, 1947.76390971, 1962.52499394, ...,
          1954.52616744, 1962.32423204, 1956.38467356],
         [1960.09187151, 1956.49718888, 1958.59004197, ...,
          1962.21782122, 1957.63611988, 1948.55803787],
         ...,
         [1958.31761227, 1961.36648752, 1965.92411018, ...,
          1961.32825257, 1957.90022838, 1953.31625482],
         [1956.99388556, 1953.54578994, 1953.86684697, ...,
          1962.70696168, 1958.61881698, 1960.93967499],
         [1953.92542898, 1955.43433005, 1960.16692688, ...,
          1962.58200314, 1955.18045557, 1949.80709673]]]),
 1: array([[[1479.66201713, 1477.87633408, 1479.40678979, ...,
          1473.94296707, 1479.32150661, 1472.69206261],
         [1473.11967918, 1489.36673663, 1483.35659768, ...,
          1481.8540894 , 1473.07942434, 1476.0525192 ],
         [1485.0510648 , 1478.11510774, 1486.3341

Finding solar energy exported, by data and split across residential and non-residential

In [31]:
n_res_solar_customers = elec_con_data.loc[:, "Number of Residential Solar Customers"]
n_non_res_solar_customers = elec_con_data.loc[:, "Number of Non-Residential Solar Customers"]

In [32]:
total_solar_generation_capacity = elec_con_data.loc[:, "Residential Generation Capacity (kWp)"] + elec_con_data.loc[0, "Non-Residential Generation Capacity (kWp)"]
p_res_solar_gen_capacity = elec_con_data.loc[:, "Residential Generation Capacity (kWp)"] / total_solar_generation_capacity
p_non_res_solar_gen_capacity = elec_con_data.loc[:, "Non-Residential Generation Capacity (kWp)"] / total_solar_generation_capacity

In [33]:
avg_res_solar_gen = elec_con_data.loc[:, "Energy exported to the grid (MWh)"] * p_res_solar_gen_capacity / 365 * 1000 / n_res_solar_customers
avg_res_solar_gen

0    7.703632
1    8.343343
2    7.792037
3    7.566992
4    3.094242
5    7.538154
6    6.412714
7    5.340947
8    2.982740
dtype: float64

Generate synthetic data

In [34]:
res_solar_X = {}
for (i, elec_con_row), avg in zip(elec_con_data.iterrows(), avg_res_solar_gen):
    res_solar_X[i] = rng.normal(
        loc=-avg,
        scale=5,
        size=(365, round(elec_con_row["Number of Residential Solar Customers"]), 1)
    ).T
res_solar_X

{0: array([[[ -8.28681184, -10.84503808,  -3.39950137, ...,  -0.96470282,
           -0.28167492,   5.53149924],
         [ -2.77886992, -14.7301761 ,  -2.20186313, ...,  -3.67831479,
           -9.01690123,  -2.83432982],
         [ -7.09752003, -16.11678845, -11.27074409, ...,  -7.08685573,
           -5.24730131, -10.15675901],
         ...,
         [-12.26967083,  -8.06039496,  -7.23035533, ..., -14.34173622,
          -10.25973389,  -2.47567844],
         [-11.10021254,  -7.64014427, -11.66524956, ...,  -4.82484354,
           -5.12836633,  -6.40677234],
         [ -3.19731413,   5.56046754,  -2.32439696, ..., -10.27860668,
            1.59598176, -12.61235153]]]),
 1: array([[[-24.43339321,  -8.15268554,  -4.89868893, ...,  -8.26092344,
          -11.54469741,  -1.94848645],
         [ -1.61393563,  -6.58123481, -10.37475963, ...,  -4.85602589,
           -4.04795865, -13.93439397],
         [-10.9661173 ,  -7.62140658, -11.59888371, ...,  -2.33517681,
           -2.90659957, -1

In [35]:
avg_non_res_solar_gen = elec_con_data.loc[:, "Energy exported to the grid (MWh)"] * p_non_res_solar_gen_capacity / 365 * 1000 / n_non_res_solar_customers
avg_non_res_solar_gen

0    49.185027
1    62.360756
2    37.315642
3    41.343036
4    10.186374
5    61.307894
6    31.095044
7    33.603408
8     8.657238
dtype: float64

Split non-residential into three groups

In [36]:
N = 3
allocation = rng.dirichlet([1.5 for i in range(N)])
mu0 = scipy.optimize.minimize(f, np.repeat(0, len(elec_con_data)), (allocation, avg_non_res_solar_gen))['x']

print(f"{allocation=}, {mu0=}: {alloc_mean(mu0, allocation)=}")
print(f"{np.array([mu0 * a for a in allocation])=}")

allocation=array([0.33658413, 0.50673033, 0.15668554]), mu0=array([147.55508199, 187.08226884, 111.94692518, 124.02910777,
        30.55912084, 183.92368075,  93.28513104, 100.81022396,
        25.97171458]): alloc_mean(mu0, allocation)=array([49.18502733, 62.36075628, 37.31564173, 41.34303592, 10.18637361,
       61.30789358, 31.09504368, 33.60340799,  8.65723819])
np.array([mu0 * a for a in allocation])=array([[49.66469925, 62.96892314, 37.67955869, 41.74622963, 10.28571518,
        61.90579251, 31.3982949 , 33.93112177,  8.74166702],
       [74.77063498, 94.8002593 , 56.72690203, 62.84931036, 15.4852333 ,
        93.19970693, 47.27040498, 51.08359777, 13.16065543],
       [23.11974776, 29.31308639, 17.54046447, 19.43356777,  4.78817236,
        28.81818131, 14.61643117, 15.79550442,  4.06939213]])


Generate synthetic data

In [37]:
non_res_solar_X = {}
for (i, elec_con_row), m0, in zip(elec_con_data.iterrows(), mu0):
    non_res_solar_X[i] = rng.normal(
        loc=np.array([-m0 * a for a in allocation]),
        scale=5,
        size=(365, round(elec_con_row["Number of Non-Residential Solar Customers"]) // N, N)
    ).T
non_res_solar_X

{0: array([[[-52.71428656, -50.19775644, -42.96281627, ..., -53.50600722,
          -56.46570121, -49.82613266],
         [-50.52304635, -45.61272261, -46.28495439, ..., -49.38360769,
          -48.42839342, -45.70634895],
         [-47.66583662, -57.0907519 , -46.19233125, ..., -42.14069145,
          -41.88407276, -51.26778762],
         ...,
         [-49.03837887, -47.67440993, -50.01448084, ..., -47.79539946,
          -45.83531603, -46.5540654 ],
         [-44.27647599, -39.16935425, -38.98932488, ..., -55.16906552,
          -45.18565837, -51.1716501 ],
         [-50.15741983, -47.45662443, -50.03588385, ..., -54.34821892,
          -55.11905329, -47.20795999]],
 
        [[-73.53887671, -65.29243831, -71.48567861, ..., -76.55935784,
          -84.11924325, -75.44106222],
         [-65.17583898, -76.17874946, -89.09381567, ..., -71.73470648,
          -77.274218  , -83.55862497],
         [-74.74683178, -71.21303799, -69.78760476, ..., -76.50932545,
          -61.20217458, -78.5

And save all synthetic data into a pickle file

In [38]:
X = {
    i: np.concatenate([x[i].reshape(-1, 365) for x in (res_X, res_solar_X, small_med_non_res_X, large_non_res_X, non_res_solar_X)])
    for i in range(len(elec_con_data))
}
with open("data/electricity_consumption_2020-2021.pkl", 'wb') as f:
    pickle.dump(X, f)

We will save weather data for each of the LGAs

First we get LGA information from https://www.olg.nsw.gov.au/public/local-government-directory/ by downloading the "All NSW Council Contact Details – XLS" file and saving to the data folder. Then we match
LGAs to those in `elec_con_data` and get the postcodes

In [39]:
lga_contacts = pd.read_excel("data/LGDGPALL.xls")
lga_contacts

,ABS,ORGNAME,POSTAL_ADD1,POSTAL_ADD2,POSTAL_SUBURB,POSTAL_STATE,POSTAL_PCODE,STREET_ADD1,STREET_ADD2,STREET_SUBURB,...,MAYOR_SAL,MAYOR_FIRST,MAYOR_LAST,MAYOR_AWARD,EMAIL,WEB,AREA,POPULATION,ABN,METRO
0,10050,Albury City Council,PO Box 323,NaN,ALBURY,NSW,2640,553 Kiewa Street,NaN,Albury,...,Clr,Kylie,King,NaN,info@alburycity.nsw.gov.au,http://www.alburycity.nsw.gov.au,306,52949,92 965 474 349,0
1,10180,Armidale Regional Council,PO Box 75A,NaN,ARMIDALE,NSW,2350,135 Rusden Street,NaN,Armidale,...,Clr,Sam,Coupland,NaN,council@armidale.nsw.gov.au,http://www.armidaleregional.nsw.gov.au,8621,30594,39 642 954 203,0
2,10250,Ballina Shire Council,PO Box 450,NaN,BALLINA,NSW,2478,40 Cherry Street,NaN,Ballina,...,Clr,Sharon,Cadwallader,NaN,council@ballina.nsw.gov.au,http://www.ballina.nsw.gov.au,485,43457,53 929 887 369,0
3,10300,Balranald Shire Council,PO Box 120,NaN,BALRANALD,NSW,2715,70 Market Street,NaN,Balranald,...,NaN,NaN,NaN,NaN,council@balranald.nsw.gov.au,http://www.balranald.nsw.gov.au,21691,2341,74 678 751 581,0
4,10470,Bathurst Regional Council,Private Mail Bag 17,NaN,BATHURST,NSW,2795,158 Russell Street,NaN,Bathurst,...,Clr,Robert,Taylor,NaN,council@bathurst.nsw.gov.au,http://www.bathurst.nsw.gov.au,3818,42779,42 173 522 302,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,18350,Wingecarribee Shire Council,PO Box 141,NaN,MOSS VALE,NSW,2577,Civic Centre,68 Elizabeth Street,Moss Vale,...,NaN,NaN,NaN,NaN,mail@wsc.nsw.gov.au,http://www.wsc.nsw.gov.au,2689,49649,49 546 344 354,0
124,18400,Wollondilly Shire Council,PO Box 21,NaN,PICTON,NSW,2571,62-64 Menangle Street,NaN,Picton,...,Clr,Matthew,Gould,NaN,council@wollondilly.nsw.gov.au,http://www.wollondilly.nsw.gov.au,2555,51002,93 723 245 808,1
125,18450,Wollongong City Council,Locked Bag 8821,NaN,WOLLONGONG DC,NSW,2500,41 Burelli Street,NaN,Wollongong,...,Clr,Gordon,Bradbery,AM,council@wollongong.nsw.gov.au,http://www.wollongong.nsw.gov.au,684,213132,63 139 525 939,0
126,18500,Woollahra Municipal Council,PO Box 61,NaN,DOUBLE BAY,NSW,1360,536 New South Head Road,NaN,Double Bay,...,Clr,Susan,Wynne,NaN,records@woollahra.nsw.gov.au,http://www.woollahra.nsw.gov.au,12,58456,32 218 483 245,1


In [40]:
idx = []
for LGA in elec_con_data['LGA']:
    orgname, score, id = process.extractOne(LGA, lga_contacts['ORGNAME'])
    idx.append(id)
    print(f"{LGA=}: {orgname=}")
idx = np.array(idx)

LGA='CENTRAL COAST*': orgname='Central Coast Council'
LGA='CESSNOCK': orgname='Cessnock City Council'
LGA='LAKE MACQUARIE': orgname='Lake Macquarie City Council'
LGA='MAITLAND': orgname='Maitland City Council'
LGA='MUSWELLBROOK': orgname='Muswellbrook Shire Council'
LGA='NEWCASTLE': orgname='Newcastle City Council'
LGA='PORT STEPHENS': orgname='Port Stephens Council'
LGA='SINGLETON': orgname='Singleton Council'
LGA='UPPER HUNTER': orgname='Upper Hunter Shire Council'


In [41]:
lga_postcodes = lga_contacts.loc[idx, "POSTAL_PCODE"]

postcode_data = pd.read_csv(
    "data/AU.txt",
    sep='\t',
    names=["country code","postal code","place name","admin name1","admin code1","admin name2","admin code2","admin name3","admin code3","latitude","longitude","accuracy"]
)
postcode_data = postcode_data.drop(postcode_data[postcode_data['postal code'].duplicated()].index)  # we will use just one of the postcodes
postcode_data

,country code,postal code,place name,admin name1,admin code1,admin name2,admin code2,admin name3,admin code3,latitude,longitude,accuracy
0,AU,200,Australian National University,Australian Capital Territory,ACT,CANBERRA,NaN,NaN,NaN,-35.2777,149.1189,1.0
1,AU,221,Barton,Australian Capital Territory,ACT,NaN,NaN,NaN,NaN,-35.3049,149.1412,4.0
2,AU,2540,Jervis Bay,Australian Capital Territory,ACT,NEW CNTRY WEST,NaN,NaN,NaN,-35.1499,150.6969,4.0
5,AU,2600,Russell,Australian Capital Territory,ACT,CANBERRA,NaN,NaN,NaN,-35.2991,149.1515,4.0
17,AU,2601,Canberra,Australian Capital Territory,ACT,CANBERRA,NaN,NaN,NaN,-35.2835,149.1281,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
16868,AU,6989,Maddington,Western Australia,WA,TANGNEY,NaN,NaN,NaN,-32.0500,115.9833,4.0
16869,AU,6990,Gosnells,Western Australia,WA,TANGNEY,NaN,NaN,NaN,-32.0810,116.0054,4.0
16870,AU,6991,Kelmscott,Western Australia,WA,TANGNEY,NaN,NaN,NaN,-32.1243,116.0259,4.0
16871,AU,6992,Armadale,Western Australia,WA,TANGNEY,NaN,NaN,NaN,-32.1461,116.0093,4.0


In [42]:
weather_data = {
    "structure": {"postcode": ["temperature_2m_max","temperature_2m_min","sunrise","sunset","precipitation_sum","precipitation_hours","shortwave_radiation_sum"]},
    "daily units": {"temperature_2m_max":"Celcius","temperature_2m_min":"Celcius","sunrise":"int","sunset":"int","precipitation_sum":"mm","precipitation_hours":"h","shortwave_radiation_sum":"MJ/m^2"}
}

def time_to_hour_int(time: str):
    dt = datetime.fromisoformat(time)
    return dt.hour * 100 + dt.minute

i = 0
for postcode in (pbar := tqdm(lga_postcodes)):
    pbar.set_postfix_str(f"{postcode=}")
    postcode_row = postcode_data[postcode == postcode_data['postal code']]
    latitude, longitude = postcode_row.latitude.item(), postcode_row.longitude.item()
    r = requests.get(f"https://archive-api.open-meteo.com/v1/era5?latitude={latitude}&longitude={longitude}&start_date=2021-07-01&end_date=2022-06-30&daily=temperature_2m_max,temperature_2m_min,sunrise,sunset,precipitation_sum,precipitation_hours,shortwave_radiation_sum&timezone=Australia%2FSydney")
    if r.ok:
        postcode_weather = r.json()
        weather_data[str(i)] = np.array([
            [temp_min, temp_max, time_to_hour_int(sunrise), time_to_hour_int(sunset)] + other_data
            for day, temp_min, temp_max, sunrise, sunset, *other_data in zip(*[v for v in postcode_weather['daily'].values()])
        ])
    else:
        tqdm.write(f"Postcode {postcode} weather data request failed")
    i += 1

with open(f"data/central_coast_hunter_weather.pkl", 'wb') as f:
    pickle.dump(weather_data, f)

100%|██████████| 9/9 [00:10<00:00,  1.17s/it, postcode=2337]


## Sampling

In [43]:
def get_customer_data(lga=0, customer=0):
    idx = np.arange(30, len(X[lga][customer]))
    expanded_idx = np.array([np.arange(i - 30, i - 1) for i in idx])
    return np.concatenate((X[lga][customer, expanded_idx].reshape(-1, 29, 1), weather_data[str(lga)][expanded_idx]), axis=-1), X[lga][customer, idx]

In [44]:
X, Y = get_customer_data()